### Match entre empresas y  freelances mediante el índice de Jaccard  

In [1]:
#librerías necesarias
import pandas as pd
import os
import sys
import numpy as np

In [25]:
# El path donde estén tus csv
# path="C://Users//lopez//Documents//cursos//master BIG DATA//TFM"
path="C://Users//Usuario//Desktop//TFM//"
#os.getcwd()
os.chdir(path)
# creamos los dataframes de los csv de compañías y freelances.
companies = pd.read_excel("companies.xlsx")
freelances = pd.read_excel("freelances.xlsx")

In [26]:
# Cogemos los datasets con los campos que vamos a necesitar
companies_cols=["Company_Name","Industry of company","Prescriptive analytics business","Speech analytics business","Predictive Analytics business",
                "Big Data Business","Machine Learning based business","Highest education","Specialization of highest education",
                "Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Engineering","Percent_skill_Engineering",
                "Percent_skill_Leadership", "Percent_skill_Business Strategy"]

companies=companies[companies_cols]

freelances_cols=["Respondent","DeveloperType","FormalEducation","MajorUndergrad","BuildingThings","LearningNewTech","RightWrongWay","CompetePeers",
                "DiversityImportant","ChallengeMyself","ChangeWorld","UnderstandComputers","InvestTimeTools"]

freelances=freelances[freelances_cols]


#### Funciones para crear los datasets finales que usaremos para aplicar el íncide de Jaccard

In [4]:
def col_to_dummies(df,col):
    """
    nos quita los NaN, no crea una categoría para ellos
    """
    df_out = pd.get_dummies(df,columns=[col], prefix=col)
    cols=df_out.columns.tolist()[1:]
    df_out[cols]=df_out[cols].astype(float)
    return df_out

In [6]:
# Funciones relacionadas con habilidades técnicas

In [7]:
def Tech_Skills_Freelancers(freelances,col_Respondent,developer_type):
    """
    """
    df_dev = freelances[[col_Respondent,developer_type]].copy()
    df_dev[developer_type]=df_dev[developer_type]
    df_dev[developer_type]=df_dev[developer_type].str.replace(';.*','')
    df_dev.loc[(df_dev[developer_type] == "Embedded applications/devices developer") | (df_dev[developer_type] == "Desktop applications developer") | (df_dev[developer_type] == "Quality assurance engineer")  ,developer_type] = "Software Engineer"
    df_dev.loc[(df_dev[developer_type] == "Machine learning Specialist"),developer_type] = "Machine Learning"
    df_dev.loc[(df_dev[developer_type] == "DevOps specialist")  ,developer_type] = "DevOps"
    df_dev.loc[(df_dev[developer_type] == "Developer with a statistics or mathematics background")  ,developer_type] = "Analytics"
    df_dev.loc[(df_dev[developer_type] == "Database administrator") | (df_dev[developer_type] == "Systems administrator") ,developer_type] = "Database or system administrator"
    df_dev.loc[(df_dev[developer_type] == "Graphics programming") | (df_dev[developer_type] == "Graphic designer") ,developer_type] = "Graphics Designer"
    df_out=col_to_dummies(df_dev,developer_type)
    return df_out

def recode_tech_skills_comp(df_companies,col_Company_Name,col_companies):
    """
    
    """
    df_aux=companies[[col_Company_Name,col_companies]].copy()
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(big data).*','Data scientist')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(analytics|cloud).*','Analytics')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(machine learning).*','Machine Learning')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(mobile).*','Mobile developer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(network|marketing|publishing|gaming|advertising).*','Graphics Designer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(software|cloud).*','software engineer')
    df_aux[col_companies]=df_aux[col_companies].str.replace('(.*|^)(?i)(E-commerce).*','Web developer')                                              
    #df_aux[col_companies]=df_aux[col_companies].fillna("sin info")
    df_aux.loc[(df_aux[col_companies] != "Data scientist") & (df_aux[col_companies] != "Analytics") &
               (df_aux[col_companies] != "Machine Learning") & (df_aux[col_companies] != "Mobile developer") &
               (df_aux[col_companies] != "Graphics Designer") & (df_aux[col_companies] != "software engineer") &
               (df_aux[col_companies] != "nan") & (df_aux[col_companies] != "Web developer"),col_companies] = "other"
    df_out=col_to_dummies(df_aux,col_companies)
    return df_out

cols_companies_DS=["Company_Name","Prescriptive analytics business","Big Data Business","Speech analytics business","Predictive Analytics business"]
cols_companies_ML=["Company_Name","Machine Learning based business"]

def recode_tech_DS_ML_comp(df_companies,cols_companies,out_col_name):
    """

    """
    df_aux=companies[cols_companies].copy()
    for i in cols_companies[1:]:
        df_aux[i]=df_aux[i].replace('Yes',1)
        df_aux[i]=df_aux[i].replace('No',0)
        df_aux[i]=df_aux[i].replace('No Info',np.nan)

    df_aux[out_col_name] = df_aux[cols_companies[1:]].max(axis=1, skipna = True)
    df_out=df_aux[[cols_companies[0],out_col_name]]
    return df_out


In [8]:
df_companies1=recode_tech_skills_comp(companies,"Company_Name","Industry of company")
df_companies2=recode_tech_DS_ML_comp(companies,cols_companies_DS,"Data scientist")
df_companies_out=pd.merge(df_companies1,df_companies2,on='Company_Name',how='outer')
df_companies3=recode_tech_DS_ML_comp(companies,cols_companies_ML,"Machine Learning")
df_companies_out1=pd.merge(df_companies_out,df_companies3,on='Company_Name',how='outer')
df_companies_out1.head()

,Company_Name,Industry of company_Analytics,Industry of company_Graphics Designer,Industry of company_Mobile developer,Industry of company_Web developer,Industry of company_other,Industry of company_software engineer,Data scientist,Machine Learning
0,Company1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Company2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Company3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Company4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Company5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
df_freelance_out1=Tech_Skills_Freelancers(freelances,"Respondent","DeveloperType")
df_freelance_out1.head()

,Respondent,DeveloperType_Analytics,DeveloperType_Data scientist,DeveloperType_Database or system administrator,DeveloperType_DevOps,DeveloperType_Graphics Designer,DeveloperType_Machine learning specialist,DeveloperType_Mobile developer,DeveloperType_Other,DeveloperType_Software Engineer,DeveloperType_Web developer
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# Funciones relacionadas con educación

In [11]:
def recode_FormalEducation(df_freelance,col_Respondent,col_freelance):
    """
    
    """
    df_aux = df_freelance[[col_Respondent,col_freelance]].copy()
    df_aux.loc[df_aux[col_freelance] == "Master's degree",col_freelance] = "Masters"
    df_aux.loc[(df_aux[col_freelance] == "Professional degree") |
               (df_aux[col_freelance] == "Bachelor's degree"),col_freelance] = "Bachelors"
    df_aux.loc[df_aux[col_freelance] == "Doctoral degree",col_freelance] = "PhD"
    df_aux.loc[df_aux[col_freelance] == "I prefer not to answer",col_freelance] = "No Info"
    df_aux.loc[(df_aux[col_freelance] == "Secondary school") |
              (df_aux[col_freelance] == "Some college/university study without earning a bachelor's degree") |
              (df_aux[col_freelance] == "Primary/elementary school") |
              (df_aux[col_freelance] == "I never completed any formal education")
               ,col_freelance] = "under bachelor's degree"
    df_out=col_to_dummies(df_aux,col_freelance)
    return df_out


def recode_MajorUndergrad(df_freelance,col_Respondent,col_freelance):
    """
    
    """
    df_aux = df_freelance[[col_Respondent,col_freelance]].copy()
    df_aux.loc[(df_aux[col_freelance] == "A social science") | (df_aux[col_freelance] == "A business discipline") |
               (df_aux[col_freelance] == "Management information systems") | (df_aux[col_freelance] == "Psychology") |
               (df_aux[col_freelance] == "Information technology, networking, or system administration") |
               (df_aux[col_freelance] == "A humanities discipline"),col_freelance] = "social science"
    df_aux.loc[(df_aux[col_freelance] == "Computer science or software engineering") |
               (df_aux[col_freelance] == "Computer programming or Web development") |
               (df_aux[col_freelance] == "Computer engineering or electrical/electronics engineering") |
               (df_aux[col_freelance] == "Mathematics or statistics"),col_freelance] = "engineering,physics, mathematics or statistics"
    df_aux.loc[df_aux[col_freelance] == "A natural science",col_freelance] = "natural science"
    df_aux.loc[df_aux[col_freelance] == "A health science",col_freelance] = "health science"
    df_aux.loc[df_aux[col_freelance] == "Fine arts or performing arts",col_freelance] = "arts"
    df_aux.loc[(df_aux[col_freelance] == "I never declared a major") |(df_aux[col_freelance] == "Something else") |
              (df_aux[col_freelance] == "A non-computer-focused engineering discipline"),col_freelance] = "other"
    df_aux[col_freelance]=df_aux[col_freelance].fillna("sin info")
    df_out=col_to_dummies(df_aux,col_freelance)
    return df_out

def recode_Specialization_education(df_companies,col_Company_Name,col_companies):
    """
    
    """
    df_aux = df_companies[[col_Company_Name,col_companies]].copy()
    df_aux[col_companies+"_clean"]=df_aux[col_companies]
    df_aux[col_companies+"_clean"]=df_aux[col_companies+"_clean"].str.replace('(.*|^)(?i)(POL.SC|bussiness|Entertainment|human|psychology|journalism|marketing|markting|communic|finance|law|information|economics|Communication|business|politic|nutrition|sociology|dsign|design|history|media|management).*','social science')
    df_aux[col_companies+"_clean"]=df_aux[col_companies+"_clean"].str.replace('(.*|^)(?i)(aerospace|artificial intelligence|mining|technology|eng|Computer|math|physics).*','engineering, physics, mathematics or statistics')
    df_aux[col_companies+"_clean"]=df_aux[col_companies+"_clean"].str.replace('(.*|^)(?i)(geology|Earth|Biology).*','natural science')
    df_aux[col_companies+"_clean"]=df_aux[col_companies+"_clean"].str.replace('(.*|^)(?i)(Biomedical|medicine|nurs).*','health science')
    df_aux[col_companies+"_clean"]=df_aux[col_companies+"_clean"].str.replace('(.*|^)(?i)(arts|music|theatre).*','arts')
    df_aux[col_companies+"_clean"]=df_aux[col_companies+"_clean"].fillna("sin info")
    df_aux.loc[(df_aux[col_companies+"_clean"] != "arts") & (df_aux[col_companies+"_clean"] != "health science") &
               (df_aux[col_companies+"_clean"] != "natural science") & (df_aux[col_companies+"_clean"] != "sin info") &
               (df_aux[col_companies+"_clean"] != "social science") &
               (df_aux[col_companies+"_clean"] != "engineering, physics, mathematics or statistics"),col_companies+"_clean"] = "other"
    df_out=col_to_dummies(df_aux[col_companies+"_clean"],col_companies)
    return df_out

In [13]:
df_companies_out2=col_to_dummies(companies[["Company_Name","Highest education"]],"Highest education")
df_companies_out2.head()

,Company_Name,Highest education_Bachelors,Highest education_Masters,Highest education_No Info,Highest education_PhD
0,Company1,0.0,1.0,0.0,0.0
1,Company2,0.0,1.0,0.0,0.0
2,Company3,1.0,0.0,0.0,0.0
3,Company4,1.0,0.0,0.0,0.0
4,Company5,1.0,0.0,0.0,0.0


In [14]:
df_companies_out3=recode_Specialization_education(companies,"Company_Name","Specialization of highest education")
df_companies_out3.head()

,Specialization of highest education_arts,"Specialization of highest education_engineering, physics, mathematics or statistics",Specialization of highest education_health science,Specialization of highest education_natural science,Specialization of highest education_other,Specialization of highest education_sin info,Specialization of highest education_social science
0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,1.0,0.0,0.0,0.0,0.0,0.0
4,0,1.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df_freelance_out2=recode_FormalEducation(freelances,"Respondent","FormalEducation")
df_freelance_out2.head()

,Respondent,FormalEducation_Bachelors,FormalEducation_Masters,FormalEducation_No Info,FormalEducation_PhD,FormalEducation_under bachelor's degree
0,1,0.0,0.0,0.0,0.0,1.0
1,2,0.0,0.0,0.0,0.0,1.0
2,3,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,1.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0


In [15]:
df_freelance_out3=recode_MajorUndergrad(freelances,"Respondent","MajorUndergrad")
df_freelance_out3.head()

,Respondent,MajorUndergrad_arts,"MajorUndergrad_engineering,physics, mathematics or statistics",MajorUndergrad_health science,MajorUndergrad_natural science,MajorUndergrad_other,MajorUndergrad_sin info,MajorUndergrad_social science
0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Funciones relacionadas con habilidades

In [18]:
# Falta definir bien los input y output en las funciones.
def recode(companies,col_percent_skill):
    """
    c=recode(companies,"Percent_skill_Business Strategy")
    c["Percent_skill_Business Strategy"].unique().tolist()
    
    """
    df_comp=companies.copy()
    df_comp[col_percent_skill]=df_comp[col_percent_skill].replace('No Info',101)
    df_comp[col_percent_skill]=df_comp[col_percent_skill].astype(float)
    df_comp.loc[(df_comp[col_percent_skill] >=0 ) & (df_comp[col_percent_skill] < 20), col_percent_skill] = 0.2
    df_comp.loc[(df_comp[col_percent_skill] >=20 ) & (df_comp[col_percent_skill] < 40), col_percent_skill] = 0.4
    df_comp.loc[(df_comp[col_percent_skill] >=40) & (df_comp[col_percent_skill] < 60), col_percent_skill] = 0.6
    df_comp.loc[(df_comp[col_percent_skill] >=60 ) & (df_comp[col_percent_skill] < 80), col_percent_skill] = 0.8
    df_comp.loc[(df_comp[col_percent_skill] >=80) & (df_comp[col_percent_skill] < 100), col_percent_skill] = 1
    df_comp.loc[(df_comp[col_percent_skill] == 101.0 ) , col_percent_skill] = "NaN"
    
    return df_comp

def recode_other_skills(df_freelance,col_freelance):
    """
    a=recodificar(freelances,"BuildingThings")
    a["BuildingThings"].unique().tolist()
    
    """
    df_aux = df_freelance.copy()
    df_aux.loc[df_aux[col_freelance] == "Strongly disagree",col_freelance] = 0.2
    df_aux.loc[df_aux[col_freelance] == "Disagree",col_freelance] = 0.4
    df_aux.loc[df_aux[col_freelance] == "Somewhat agree",col_freelance] = 0.6
    df_aux.loc[df_aux[col_freelance] == "Agree",col_freelance] = 0.8
    df_aux.loc[df_aux[col_freelance] == "Strongly agree",col_freelance] = 1
    
    return df_aux

In [19]:
a=recode_other_skills(freelances,"BuildingThings")
b=recode_other_skills(a,"LearningNewTech")
c=recode_other_skills(b,"RightWrongWay")
d=recode_other_skills(c,"CompetePeers")
e=recode_other_skills(d,"DiversityImportant")
f=recode_other_skills(e,"ChallengeMyself")
g=recode_other_skills(f,"UnderstandComputers")
h=recode_other_skills(g,"InvestTimeTools")
df_Freelancers=recode_other_skills(h,"ChangeWorld")

In [20]:
def Skill_leadership():
    df_Freelancers['Skill_leadership'] = df_Freelancers[["RightWrongWay", "CompetePeers","DiversityImportant","ChallengeMyself"]].sum(axis=1, skipna = True)
    df_Freelancers['Percent_Skill_leadership']=df_Freelancers['Skill_leadership']/4
    return df_Freelancers

def Skill_Operations():
    df_Freelancers['Skill_Operations'] = df_Freelancers[["UnderstandComputers", "InvestTimeTools"]].sum(axis=1, skipna = True)
    df_Freelancers['Percent_Skill_Operations']=df_Freelancers['Skill_Operations']/2
    return df_Freelancers

Skill_leadership()
Skill_Operations()

,Respondent,DeveloperType,FormalEducation,MajorUndergrad,BuildingThings,LearningNewTech,RightWrongWay,CompetePeers,DiversityImportant,ChallengeMyself,ChangeWorld,UnderstandComputers,InvestTimeTools,Skill_leadership,Percent_Skill_leadership,Skill_Operations,Percent_Skill_Operations
0,1,NaN,Secondary school,NaN,1,0.8,0.6,0.4,0.8,0.8,0.8,0.4,1,2.6,0.65,1.4,0.7
1,2,NaN,Some college/university study without earning ...,Computer science or software engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.0
2,3,Other,Bachelor's degree,Computer science or software engineering,1,1,0.4,0.4,1,0.8,0.8,0.4,0.6,2.6,0.65,1.0,0.5
3,4,NaN,Doctoral degree,A non-computer-focused engineering discipline,1,1,0.6,0.6,0.8,1,0.8,0.2,0.8,3.0,0.75,1.0,0.5
4,5,Mobile developer; Graphics programming; Deskto...,Master's degree,Computer science or software engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51387,51388,Web developer; Developer with a statistics or ...,Bachelor's degree,A social science,0.8,1,0.4,0.4,1,0.8,0.6,0.8,1,2.6,0.65,1.8,0.9
51388,51389,NaN,Master's degree,Computer programming or Web development,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.0
51389,51390,Web developer; Systems administrator,Some college/university study without earning ...,"Information technology, networking, or system ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.0
51390,51391,Web developer; Mobile developer,Bachelor's degree,Computer science or software engineering,0.8,1,0.8,0.4,0.4,1,0.2,0.2,1,2.6,0.65,1.2,0.6


In [27]:
var_comp_recode=["Percent_skill_Entrepreneurship","Percent_skill_Operations","Percent_skill_Leadership"]
var_freelances_recode=["BuildingThings","LearningNewTech", "RightWrongWay","CompetePeers", "DiversityImportant","ChallengeMyself","UnderstandComputers","InvestTimeTools","ChangeWorld"]

for i in var_comp_recode:
    companies=recode(companies,i)
    companies_dataset=companies
for x in var_freelances_recode:
    freelances=recode_other_skills(freelances,x)
    freelances_dataset= freelances
    

In [28]:
freelances_dataset.head()

,Respondent,DeveloperType,FormalEducation,MajorUndergrad,BuildingThings,LearningNewTech,RightWrongWay,CompetePeers,DiversityImportant,ChallengeMyself,ChangeWorld,UnderstandComputers,InvestTimeTools
0,1,NaN,Secondary school,NaN,1,0.8,0.6,0.4,0.8,0.8,0.8,0.4,1
1,2,NaN,Some college/university study without earning ...,Computer science or software engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Other,Bachelor's degree,Computer science or software engineering,1,1,0.4,0.4,1,0.8,0.8,0.4,0.6
3,4,NaN,Doctoral degree,A non-computer-focused engineering discipline,1,1,0.6,0.6,0.8,1,0.8,0.2,0.8
4,5,Mobile developer; Graphics programming; Deskto...,Master's degree,Computer science or software engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Generamos los dataset finales

In [ ]:
freelances_dataset.to_excel("C://Users//Usuario//Desktop//TFM//freelances_final.xlsx", index=True)
companies_dataset.to_excel("C://Users//Usuario//Desktop//TFM//companies_final.xlsx", index=True)

In [22]:
# Aplicación del índice de Jaccard 

from sklearn.metrics import jaccard_score
import numpy as np

# Ejemplo
comp_names=["c1","c2","c3","c4"]
freelances_names=["f1","f2","f3"]
companies=np.array([[0,1,2,3,1,2],[1,2,1,1,4,1],[1,0,0,0,2,4],[1,1,1,1,1,1]])
freelances=np.array([[0,0,0,3,1,2],[0,1,0,1,2,1],[0,0,2,9,5,4]])
#Buscando todas las combinaciones posibles
tuples = []
#compania_t=companias.index.values
lista=[]

for i in range(len(companies)):
    for j in range(len(freelances)):
        comp_x=companies
        free_y=freelances
        jaccard=jaccard_score(comp_x[i], free_y[j],average="weighted")
        lista.append(jaccard)
        tuples.append([comp_names[i],freelances_names[j]])
                      
df_match=pd.DataFrame({'company-freelance' : tuples, 'score':lista})
print(df_match)

   company-freelance     score
0           [c1, f1]  0.555556
1           [c1, f2]  0.166667
2           [c1, f3]  0.250000
3           [c2, f1]  0.000000
4           [c2, f2]  0.266667
5           [c2, f3]  0.000000
6           [c3, f1]  0.250000
7           [c3, f2]  0.291667
8           [c3, f3]  0.291667
9           [c4, f1]  0.166667
10          [c4, f2]  0.500000
11          [c4, f3]  0.000000


In [ ]:
companias= pd.read_excel(open("C://Users//Usuario//Desktop//TFM//Companias.xlsx",'rb'),index_col=0)
freelances= pd.read_excel(open("C://Users//Usuario//Desktop//TFM//Frelancer.xlsx",'rb'), index_col=0)

#Para una fila 
from sklearn.metrics import jaccard_similarity_score
comp=companias.iloc[0]
free=freelances.iloc[0]
jaccard_similarity_score(comp, free)
print()

#Iterando en fila 1 con fila 1, fila 2 con fila 2 

#Buscando todas las combinaciones posibles
import itertools
num = [0,1,2,3,4,5]
tuples = []
compania_t=companias.index.values

res = itertools.combinations(num, 2)
for e in res:
    tuples.append(e)
        

lista=[]

for i in range(len(tuples)):
    x=tuples[i][0]
    y=tuples[i][1]
    comp_x=companias.iloc[x]
    free_y=freelances.iloc[y]
    jaccard=jaccard_similarity_score(comp_x, free_y)
    lista.append(jaccard)
    
df_match=pd.DataFrame({'tuplas': tuples, 'score':lista})
print(df_match)